In [33]:
import re
from utils import read_lines

reg = re.compile(r'(\w+) x=(-?\d+)..(-?\d+),y=(-?\d+)..(-?\d+),z=(-?\d+)..(-?\d+)')
def parse_line(line):
    m = re.match(reg, line)
    return m.group(1), [[int(m.group(2)), int(m.group(3))], [int(m.group(4)), int(m.group(5))], [int(m.group(6)), int(m.group(7))]]

def parse_input(input_file):
    return [parse_line(line) for line in read_lines(input_file)]

def part1(input_file):
    commands = parse_input(input_file)
    matrix = [[[0] * 101 for _ in range(101)] for _ in range(101)]
    for cmd, [[x1, x2], [y1, y2], [z1, z2]] in commands:
        if x1 < -50 or x1 > 50:
            break
        v = 1 if cmd == 'on' else 0

        for x in range(x1 + 50, x2 + 51):
            for y in range(y1 + 50, y2 + 51):
                for z in range(z1 + 50, z2 + 51):
                    matrix[x][y][z] = v
    return sum(sum(sum(z) for z in yz) for yz in matrix)

def intersect(low, up, low1, up1):
    # return (intersect flag, non intersect part(s), intersect part)
    if up < low1 or low > up1:
        return False, [], []
    
    ans = []
    if low < low1:
        ans.append([low, low1 - 1])
    if up > up1:
        ans.append([up1 + 1, up])
    return True, ans, [max(low, low1), min(up, up1)]


def cube_subtract(cube1, cube2):
    # exclude cube1 and cube2 intersection part from cube1
    [[x_low1, x_up1], [y_low1, y_up1], [z_low1, z_up1]] = cube1
    [[x_low2, x_up2], [y_low2, y_up2], [z_low2, z_up2]] = cube2
    is_x_inter, x_exclude, x_inter = intersect(x_low1, x_up1, x_low2, x_up2)
    is_y_inter, y_exclude, y_inter = intersect(y_low1, y_up1, y_low2, y_up2)
    is_z_inter, z_exclude, z_inter = intersect(z_low1, z_up1, z_low2, z_up2)
    if not (is_x_inter and is_y_inter and is_z_inter):
        return [cube1]
    ans = []
    for x_ex in x_exclude:
        ans.append([x_ex, [y_low1, y_up1], [z_low1, z_up1]])
    for y_ex in y_exclude:
        ans.append([x_inter, y_ex, [z_low1, z_up1]])
    for z_ex in z_exclude:
        ans.append([x_inter, y_inter, z_ex])
    return ans

def calc_on_count(cube):
    [[x_low, x_up], [y_low, y_up], [z_low, z_up]] = cube
    return (x_up - x_low + 1) * (y_up - y_low + 1) * (z_up - z_low + 1)

def part2(input_file):
    commands = parse_input(input_file)
    on_cubes = []
    for cmd, cube in commands:
        
        if cmd == 'on':
            remain_cubes = [cube] # substract all current on cubes, the remain will be add to on_cubes
            for cube2 in on_cubes:
                new_remain_cubes = []
                for cube1 in remain_cubes:
                    for x in cube_subtract(cube1, cube2):
                        new_remain_cubes.append(x)
                remain_cubes = new_remain_cubes
                if not remain_cubes:
                    break
            for x in remain_cubes:
                on_cubes.append(x)
        else:
             # each current on_cubes substrct off cubes, what's left will be in new on_cubes
            new_on_cubes = []
            for cube1 in on_cubes:
                for x in cube_subtract(cube1, cube):
                    new_on_cubes.append(x)
            on_cubes = new_on_cubes
    return sum(calc_on_count(cube) for cube in on_cubes)



In [22]:
part1('inputs/day22_test.txt')

590784

In [23]:
part1('inputs/day22.txt')

588120

In [34]:
part2('inputs/day22_test2.txt')

2758514936282235

In [35]:
part2('inputs/day22.txt')

1134088247046731